In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
import re
import urllib.parse

# Create a new instance of the chrome driver
driver = webdriver.Chrome()

# Go to the website
driver.get("https://www.wongnai.com/recipes/tags/main-dish-recipes?sort.type=1")


In [ ]:
# Locate the "show more" button
show_more_button = driver.find_element(By.CSS_SELECTOR, "button.sc-AxiKw.rxj62z-1.inRrnP")
count = 0
# Scroll down the page and click the "show more" button until all the desired data is loaded
while show_more_button.is_displayed():
    driver.execute_script("arguments[0].scrollIntoView();", show_more_button)
    show_more_button.click()
    sleep(5)
    count+=1
    if count > 700:
      break



In [ ]:
# Extract the HTML data
html_data = driver.page_source

# Close the driver
driver.quit()

In [ ]:
# Use BeautifulSoup to parse the HTML data
soup = BeautifulSoup(html_data, 'html.parser')

# Find all links in the HTML data
links = soup.find_all('a')

# Create a list to store the links that start with '/recipes'
recipe_url_ugc = set()
recipe_url = set()
base = 'https://www.wongnai.com/'
# Iterate over the links and check if each one starts with '/recipes'
for link in links:
    #print(link)
    # if re.match(r'(\/?)recipes/ugc',link.get('href')):
    #     recipe_links_ugc.add(link.get('href'))
    if re.match(r'(\/?)recipes/ugc',link.get('href')):
        recipe_url_ugc.add(urllib.parse.urljoin(base,link.get('href')))
    elif re.match(r'(\/?)recipes',link.get('href')):
        recipe_url.add(urllib.parse.urljoin(base,link.get('href')))

# Print the links
#print(list(recipe_links))


In [ ]:
# with open("recipes.txt", "w") as file:
#     file.write("\n".join(list(recipe_links)))
# with open("recipes_ugc.txt", "w") as file:
#     file.write("\n".join(list(recipe_links_ugc)))
with open("recipes_url_ugc.txt", "w") as file:
    file.write("\n".join(list(recipe_url_ugc)))
with open("recipes_url.txt", "w") as file:
    file.write("\n".join(list(recipe_url)))

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def get_recipe_data_ugc(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    try:
        script = soup.find("script", attrs={"type": "text/javascript"}).text
        start = script.find("window._wn = ") + len("window._wn = ")
        end = script.find("};", start) + 1
        json_str = script[start:end]
        data = json.loads(json_str)
        return data["store"]["recipeData"]["value"]["data"]
    except Exception as e:
        print(e)



In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def get_recipe_data(url):
    headers = {'User-Agent': 'PostmanRuntime/7.26.10'}

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    try:

        script = soup.find("script", attrs={"type": "text/javascript"}).text
        start = script.find("window._wn = ") + len("window._wn = ")
        end = script.find("};", start) + 1
        json_str = script[start:end]
        data = json.loads(json_str)
        return data["store"]["article"]["value"]
    except Exception as e:
        print(e)

In [ ]:
def replace_all(obj, find, replace):
    if isinstance(obj, list):
        for i in range(len(obj)):
            obj[i] = replace_all(obj[i], find, replace)
    elif isinstance(obj, dict):
        for k, v in obj.items():
            obj[k] = replace_all(v, find, replace)
    elif isinstance(obj, str):
        obj = obj.replace(find, replace)
    return obj

In [ ]:
filename = "recipe_data_ugc.json"
f = open("../data/recipes_url_ugc.txt","r")
recipe_url_ugc = f.read().split('\n')
try:
    with open(filename, "r") as f:
        old_data = json.load(f)
except FileNotFoundError:
    old_data = []
# url = "https://www.wongnai.com/recipes/ugc/008cc13d05504d2f86e059f4609224fd"
recipe_url_ugc = list(recipe_url_ugc)[len(old_data)+1:]

for url in recipe_url_ugc:
    new_data = get_recipe_data_ugc(url)
    new_data = replace_all(new_data, "_-x_", "1968x0")
    # updated_data = old_data + [new_data]
    if(new_data):
        old_data.append(new_data)
    else:
        break
    
with open(filename, "w") as f:
    json.dump(old_data, f)

print("Data written to file:", filename)

In [ ]:
filename = "recipe_data.json"

try:
    with open(filename, "r") as f:
        old_data = json.load(f)
except FileNotFoundError:
    old_data = []
# url = "https://www.wongnai.com/recipes/ugc/008cc13d05504d2f86e059f4609224fd"

for url in recipe_url:
    new_data = get_recipe_data(url)
    new_data = replace_all(new_data, "_-x_", "1968x0")
    # updated_data = old_data + [new_data]
    old_data.append(new_data)
    
with open(filename, "w") as f:
    json.dump(old_data, f)

print("Data written to file:", filename)

In [ ]:
def clean_data_ugc(recipe_data:dict):
  recipe_data = [d for d in recipe_data if d != None] 
  remove_keys = ["id", "description","yield","cookTimeH","cookTimeM","prepTimeH","prepTimeM","state","quality","type","createdTime","lastUpdatedTime","publishedTime","totalHomework","totalComment","owner","instructions","bookmark","videos","orderLink"]
  list(map(recipe_data.pop, remove_keys))
  print(recipe_data)

In [ ]:
from boltons.iterutils import remap

def sof_solution(data):
    sensitive = {"id", "description","yield","cookTimeH","cookTimeM","prepTimeH","prepTimeM","state","quality","type","createdTime","lastUpdatedTime","publishedTime","totalHomework","totalComment","owner","instructions","bookmark","videos","orderLink","photos.photoId",
    "photos.width","photos.height","photos.description","photos.thumbnailUrl","photos.seq","ingredients.id","ingredients.recipeId",
    "ingredients.ingredientThaiName","ingredients.ingredientEnglishName","ingredients.seq","tags.id","tags.thaiName",
    "tags.englishName"}

    clean = remap(
        data,
        visit=lambda path, key, value: drop_keys(path, key, value, sensitive)
    )
    for idx,x in enumerate(clean["tags"]):
        clean["tags"][idx]["category"] = x["category"]["primaryName"]
    return clean

def drop_keys(path, key, value, sensitive):
    if len(path) > 0:
        nested_key = f"{'.'.join([i for i in path if isinstance(i, str)])}.{key}"
        return nested_key not in sensitive
    return key not in sensitive

In [ ]:
import json
f = open ('recipe_data_ugc.json', "r",encoding="utf-8")
data = json.loads(f.read())

with open("clean_recipe_data_ugc.json","w") as f:
  json.dump([sof_solution(d) for d in data],f,ensure_ascii=False)

In [ ]:
keywordspicy = ["พริกขี้หนู","พริกแกง","พริกชี้ฟ้า","น้ำมันพริกเผา","น้ำพริกเผา","พริกจินดา","พริกป่น","พริกแห้ง"]